In [1]:
!pip3 install beautifulsoup4

In [2]:
import pandas as pd
import time
from bs4 import BeautifulSoup
from urllib import request
import re

In [3]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException

## Amazon France

In [4]:
def recuperer_infos_par_page_amazon_france(nb_page):  
    all_products=[]
    for i in range(1,nb_page+1):
        url = 'https://www.amazon.fr/gp/bestsellers/2021/dmusic/digital-music-track/ref=zg_bsar_pg_'+str(i)+'/ref=zg_bsar_pg_'+str(i)+'?ie=UTF8&pg='+str(i)
         
        options = webdriver.ChromeOptions()
        options.add_argument('-headless')
        driver = webdriver.Chrome('chromedriver', options = options) 
        
        
        wait = WebDriverWait(driver, 20)
        driver.get(url)
       
        try:
            wait.until(EC.element_to_be_clickable((By.ID, "sp-cc-accept"))).click()
        except  TimeoutException:
            pass
        screen_height = driver.execute_script("return window.screen.height;")   # get the screen height of the web
        i = 1
        
        while True:
            # scroll one screen height each time
            driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))  
            i += 1
            time.sleep(5)
            # update scroll height each time after scrolled, as the scroll height can change after we scrolled the page
            scroll_height = driver.execute_script("return document.body.scrollHeight;")  
            # Break the loop when the height we need to scroll to is larger than the total scroll height
            if (screen_height) * i > scroll_height:
                break
                
        html = driver.page_source
        soup = BeautifulSoup(html,'html.parser')
        

        for d in soup.findAll('div', attrs={'id':'gridItemRoot'}):
            one_product=[]
            classement= d.find('span', attrs={'class':'zg-bdg-text'})
            titre_one = d.find('div', attrs={'class':'_cDEzb_p13n-sc-css-line-clamp-3_g3dy1'})
            titre_reste = d.find('div', attrs={'class':'_cDEzb_p13n-sc-css-line-clamp-2_EWgCb'})
            artiste=d.find('div', attrs={'class':'_cDEzb_p13n-sc-css-line-clamp-1_1Fn1y'})
            image=d.find('img', attrs={'class':'a-dynamic-image p13n-sc-dynamic-image p13n-product-image'})
            note=d.find('div', attrs={'class':'a-icon-row'})
            
            
            one_product.append(classement.text)
            if(titre_one!=None):
                one_product.append(titre_one.text)
            else:
                one_product.append(titre_reste.text)
            one_product.append(artiste.text.title())
            one_product.append(image.attrs['src'])
            one_product.append(note.a.attrs['title'])
            one_product.append("Amazon_fr")
            one_product.append("France")
        
            all_products.append(one_product)
        
    return all_products

In [5]:
data_amazon_fr=pd.DataFrame(recuperer_infos_par_page_amazon_france(2))
data_amazon_fr.columns =['Classement', 'Titre', 'Artiste', 'Image','Avis','Site','Region']

In [6]:
data_amazon_fr['Classement']=data_amazon_fr['Classement'].str[1:]
data_amazon_fr['Avis']=data_amazon_fr['Avis'].str[:3]

In [7]:
data_amazon_fr

,Classement,Titre,Artiste,Image,Avis,Site,Region
0,1,Derrière le brouillard,Grand Corps Malade & Louane,https://images-eu.ssl-images-amazon.com/images...,"4,7",Amazon_fr,France
1,2,Jerusalema (feat. Nomcebo Zikode),Master Kg,https://images-eu.ssl-images-amazon.com/images...,"4,7",Amazon_fr,France
2,3,"1,2,3",Amel Bent & Hatik,https://images-eu.ssl-images-amazon.com/images...,"4,7",Amazon_fr,France
3,4,Iko Iko (My Bestie),Justin Wellington Feat. Small Jam,https://images-eu.ssl-images-amazon.com/images...,"4,6",Amazon_fr,France
4,5,Bad Habits,Ed Sheeran,https://images-eu.ssl-images-amazon.com/images...,"4,5",Amazon_fr,France
...,...,...,...,...,...,...,...
90,96,Danser encore,Hk,https://images-eu.ssl-images-amazon.com/images...,"4,9",Amazon_fr,France
91,97,Levitating,Dua Lipa,https://images-eu.ssl-images-amazon.com/images...,"4,7",Amazon_fr,France
92,98,Control,Zoe Wees,https://images-eu.ssl-images-amazon.com/images...,"4,8",Amazon_fr,France
93,99,Perfect,Ed Sheeran,https://images-eu.ssl-images-amazon.com/images...,"4,7",Amazon_fr,France


## SNEP France

In [8]:
url = 'https://snepmusique.com/les-tops/le-top-de-lannee/top-singles-annee/?annee=2021'
request_text = request.urlopen(url).read()

soup = BeautifulSoup(request_text, 'html.parser')

In [9]:
def recuperer_infos_snep(soup):  
    all_products=[]
    
    for d in soup.findAll('div', attrs={'class':'item'}):
        one_product=[]
        if (len(all_products)<100):
            classement= d.find('div', attrs={'class':'rang'})
            titre = d.find('div', attrs={'class':'titre'})
            artiste=d.find('div', attrs={'class':'artiste'})
            label=d.find('div', attrs={'class':'editeur'})
            image=d.find('div', attrs={'class':'cover'})
            
            #.title() sert a mettre une majuscule au debut des noms
            one_product.append(classement.text)
            one_product.append(titre.text.title())
            one_product.append(artiste.text.title())
            one_product.append(label.text.title())
            one_product.append(image.attrs['style'])
            one_product.append("Snep")
            one_product.append("France")
        
            all_products.append(one_product)
        
    return all_products
recuperer_infos_snep(soup)

[['1',
  'La Kiffance',
  'Naps',
  'Believe',
  'background:url(https://images.music-story.com/img/album_N/1688597-naps-la-kiffance.jpeg)',
  'Snep',
  'France'],
 ['2',
  'Petrouchka',
  'Soso Maness',
  'Rca Group',
  'background:url(https://images.music-story.com/img/album_S/1702150-soso-maness-petrouchka.jpeg)',
  'Snep',
  'France'],
 ['3',
  'Tdb',
  'Oboy',
  'Believe',
  'background:url(https://images.music-story.com/img/album_O/1705799-oboy-tdb.jpeg)',
  'Snep',
  'France'],
 ['4',
  'Bande Organisée',
  'Kofs, Soso Maness, Elams, Jul, Sch, Naps, Solda, Houari, 13 Organisée',
  'Believe',
  'background:url(https://images.music-story.com/img/album_/1662798--bande-organisee.jpeg)',
  'Snep',
  'France'],
 ['5',
  'Montero (Call Me By Your Name)',
  'Lil Nas X',
  'Sony Music Entertainment',
  'background:url(https://images.music-story.com/img/album_/1693053--montero-call-me-by-your-name.jpeg)',
  'Snep',
  'France'],
 ['6',
  'Save Your Tears',
  'The Weeknd',
  'Island Def Jam

In [10]:
data_snep_fr=pd.DataFrame(recuperer_infos_snep(soup))
data_snep_fr.columns =['Classement', 'Titre','Artiste', 'Label', 'Image','Site','Region']

In [11]:
#recuperer ce qu'il y a entre les parantheses
data_snep_fr['Image']=data_snep_fr['Image'].apply(lambda x : x[ x.find( '(' )+1 : x.find( ')' ) ])
data_snep_fr['Image'].iloc[0]

'https://images.music-story.com/img/album_N/1688597-naps-la-kiffance.jpeg'

In [12]:
data_snep_fr

,Classement,Titre,Artiste,Label,Image,Site,Region
0,1,La Kiffance,Naps,Believe,https://images.music-story.com/img/album_N/168...,Snep,France
1,2,Petrouchka,Soso Maness,Rca Group,https://images.music-story.com/img/album_S/170...,Snep,France
2,3,Tdb,Oboy,Believe,https://images.music-story.com/img/album_O/170...,Snep,France
3,4,Bande Organisée,"Kofs, Soso Maness, Elams, Jul, Sch, Naps, Sold...",Believe,https://images.music-story.com/img/album_/1662...,Snep,France
4,5,Montero (Call Me By Your Name),Lil Nas X,Sony Music Entertainment,https://images.music-story.com/img/album_/1693...,Snep,France
...,...,...,...,...,...,...,...
95,96,Ratpi World,Booba,Capitol Music France,https://images.music-story.com/img/album_B/168...,Snep,France
96,97,Whoopty,Cj,Warner,https://images.music-story.com/img/album_/1672...,Snep,France
97,98,Easy On Me,Adele,Sony Music Entertainment,https://images.music-story.com/img/album_A/171...,Snep,France
98,99,Coffre Plein,Koba Lad,Capitol Music France,https://images.music-story.com/img/album_K/167...,Snep,France


## Amazon Monde 

In [13]:
def recuperer_infos_par_page_amazon_monde(nb_page):  
    all_products=[]
    for i in range(1,nb_page+1):
        url = 'https://www.amazon.com/gp/bestsellers/2021/dmusic/digital-music-track/ref=zg_bsar_pg_'+str(i)+'/ref=zg_bsar_pg_'+str(i)+'?ie=UTF8&pg='+str(i)
        
        options = webdriver.ChromeOptions()
        options.add_argument('-headless')
        driver = webdriver.Chrome('chromedriver', options = options) 
        
        wait = WebDriverWait(driver, 20)
        driver.get(url)
       
        try:
            wait.until(EC.element_to_be_clickable((By.ID, "sp-cc-accept"))).click()
        except  TimeoutException:
            pass
        WebDriverWait(driver, 10)
        
        screen_height = driver.execute_script("return window.screen.height;")   # get the screen height of the web
        i = 1
        
        while True:
            # scroll one screen height each time
            driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))  
            i += 1
            time.sleep(5)
            # update scroll height each time after scrolled, as the scroll height can change after we scrolled the page
            scroll_height = driver.execute_script("return document.body.scrollHeight;")  
            # Break the loop when the height we need to scroll to is larger than the total scroll height
            if (screen_height) * i > scroll_height:
                break
                
        html = driver.page_source
        soup = BeautifulSoup(html,'html.parser')

        for d in soup.findAll('div', attrs={'id':'gridItemRoot'}):
            one_product=[]
            classement= d.find('span', attrs={'class':'zg-bdg-text'})
            titre_one = d.find('div', attrs={'class':'_cDEzb_p13n-sc-css-line-clamp-3_g3dy1'})
            titre_reste = d.find('div', attrs={'class':'_cDEzb_p13n-sc-css-line-clamp-2_EWgCb'})
            artiste=d.find('div', attrs={'class':'_cDEzb_p13n-sc-css-line-clamp-1_1Fn1y'})
            image=d.find('img', attrs={'class':'a-dynamic-image p13n-sc-dynamic-image p13n-product-image'})
            note=d.find('div', attrs={'class':'a-icon-row'})
            
            one_product.append(classement.text)
            if(titre_one!=None):
                one_product.append(titre_one.text)
            else:
                one_product.append(titre_reste.text)
            one_product.append(artiste.text.title())
            one_product.append(image.attrs['src'])
            if(note!=None):
                one_product.append(note.a.attrs['title'])
            else:
                one_product.append("")
            one_product.append("Amazon_com")
            one_product.append("Monde")
                
            all_products.append(one_product)
            
        
    return all_products

In [14]:
data_amazon_world=pd.DataFrame(recuperer_infos_par_page_amazon_monde(2))
data_amazon_world.columns =['Classement', 'Titre', 'Artiste', 'Image','Avis','Site','Region']

In [15]:
data_amazon_world['Classement']=data_amazon_world['Classement'].str[1:]
data_amazon_world['Avis']=data_amazon_world['Avis'].str[:3]

In [16]:
data_amazon_world

,Classement,Titre,Artiste,Image,Avis,Site,Region
0,1,Easy On Me,Adele,https://images-na.ssl-images-amazon.com/images...,4.8,Amazon_com,Monde
1,2,Shivers,Ed Sheeran,https://images-na.ssl-images-amazon.com/images...,4.9,Amazon_com,Monde
2,3,Somewhere Over The Rainbow/What A Wonderful World,Israel Kamakawiwo'Ole,https://images-na.ssl-images-amazon.com/images...,4.9,Amazon_com,Monde
3,4,Cold Heart (PNAU Remix),Elton John & Dua Lipa,https://images-na.ssl-images-amazon.com/images...,4.9,Amazon_com,Monde
4,5,Over the Rainbow,Israel Kamakawiwo'Ole,https://images-na.ssl-images-amazon.com/images...,4.9,Amazon_com,Monde
...,...,...,...,...,...,...,...
92,94,All Of Me,John Legend,https://images-na.ssl-images-amazon.com/images...,4.7,Amazon_com,Monde
93,95,Lose [Explicit],Ksi & Lil Wayne,https://images-na.ssl-images-amazon.com/images...,5.0,Amazon_com,Monde
94,96,Uptown Funk [Explicit],Mark Ronson Feat. Bruno Mars,https://images-na.ssl-images-amazon.com/images...,4.8,Amazon_com,Monde
95,97,Therefore I Am,Billie Eilish,https://images-na.ssl-images-amazon.com/images...,4.9,Amazon_com,Monde


## Billboard

In [17]:
url = 'https://www.billboard.com/charts/year-end/2021/billboard-global-200/'
request_text = request.urlopen(url).read()

soup = BeautifulSoup(request_text,'html.parser')

In [18]:
def recuperer_infos_billboard(soup):  
    all_products=[]
    
    for d in soup.findAll('div', attrs={'class':'o-chart-results-list-row-container'}):
        one_product=[]
        if (len(all_products)<100):
            classement= d.find('span', attrs={'class':'u-letter-spacing-0080@tablet'})
            titre = d.find('h3', attrs={'id':'title-of-a-story'})
            artiste=d.find('span', attrs={'class':'lrv-u-display-block'})
            
            #image=d.find('div', attrs={'class':'lrv-a-crop-1x1 a-crop-67x100@mobile-max'})
            image=d.find('div', attrs={'class':'a-crop-67x100@mobile-max'})
        
            
            one_product.append(classement.text.strip())
            one_product.append(titre.text.strip().title())
            one_product.append(artiste.text.strip().title())
            one_product.append(image.img.attrs['data-lazy-src'])
            one_product.append("Billboard")
            one_product.append("Monde")
        
            all_products.append(one_product)
        
    return all_products
recuperer_infos_billboard(soup)

[['1',
  'Levitating',
  'Dua Lipa',
  'https://charts-static.billboard.com/img/2020/04/dua-lipa-eqf-levitating-tr9-180x180.jpg',
  'Billboard',
  'Monde'],
 ['2',
  'Save Your Tears',
  'The Weeknd & Ariana Grande',
  'https://charts-static.billboard.com/img/2020/12/the-weeknd-xmx-save-your-tears-e5b-180x180.jpg',
  'Billboard',
  'Monde'],
 ['3',
  'Blinding Lights',
  'The Weeknd',
  'https://charts-static.billboard.com/img/2019/12/the-weeknd-xmx-blinding-lights-q48-180x180.jpg',
  'Billboard',
  'Monde'],
 ['4',
  'Drivers License',
  'Olivia Rodrigo',
  'https://charts-static.billboard.com/img/2021/01/olivia-rodrigo-3wl-drivers-license-w08-180x180.jpg',
  'Billboard',
  'Monde'],
 ['5',
  'Dynamite',
  'Bts',
  'https://charts-static.billboard.com/img/2020/09/bts-hm9-dynamite-zg6-180x180.jpg',
  'Billboard',
  'Monde'],
 ['6',
  'Dakiti',
  'Bad Bunny & Jhay Cortez',
  'https://charts-static.billboard.com/img/2020/11/bad-bunny-c3m-dakiti-wxy-180x180.jpg',
  'Billboard',
  'Monde']

In [19]:
data_billboard_world=pd.DataFrame(recuperer_infos_billboard(soup))
data_billboard_world.columns =['Classement', 'Titre', 'Artiste', 'Image','Site','Region']

In [20]:
data_billboard_world

,Classement,Titre,Artiste,Image,Site,Region
0,1,Levitating,Dua Lipa,https://charts-static.billboard.com/img/2020/0...,Billboard,Monde
1,2,Save Your Tears,The Weeknd & Ariana Grande,https://charts-static.billboard.com/img/2020/1...,Billboard,Monde
2,3,Blinding Lights,The Weeknd,https://charts-static.billboard.com/img/2019/1...,Billboard,Monde
3,4,Drivers License,Olivia Rodrigo,https://charts-static.billboard.com/img/2021/0...,Billboard,Monde
4,5,Dynamite,Bts,https://charts-static.billboard.com/img/2020/0...,Billboard,Monde
...,...,...,...,...,...,...
95,96,Pareja Del Ano,Sebastian Yatra X Myke Towers,https://charts-static.billboard.com/img/2021/0...,Billboard,Monde
96,97,Anyone,Justin Bieber,https://charts-static.billboard.com/img/2021/0...,Billboard,Monde
97,98,Prisoner,Miley Cyrus Featuring Dua Lipa,https://charts-static.billboard.com/img/2020/1...,Billboard,Monde
98,99,Woman,Doja Cat,https://charts-static.billboard.com/img/2018/0...,Billboard,Monde


In [21]:
result = pd.concat([data_amazon_fr, data_snep_fr, data_amazon_world, data_billboard_world], ignore_index=True, join='outer', sort=False)

In [22]:
result = result.fillna('')

In [23]:
result

,Classement,Titre,Artiste,Image,Avis,Site,Region,Label
0,1,Derrière le brouillard,Grand Corps Malade & Louane,https://images-eu.ssl-images-amazon.com/images...,"4,7",Amazon_fr,France,
1,2,Jerusalema (feat. Nomcebo Zikode),Master Kg,https://images-eu.ssl-images-amazon.com/images...,"4,7",Amazon_fr,France,
2,3,"1,2,3",Amel Bent & Hatik,https://images-eu.ssl-images-amazon.com/images...,"4,7",Amazon_fr,France,
3,4,Iko Iko (My Bestie),Justin Wellington Feat. Small Jam,https://images-eu.ssl-images-amazon.com/images...,"4,6",Amazon_fr,France,
4,5,Bad Habits,Ed Sheeran,https://images-eu.ssl-images-amazon.com/images...,"4,5",Amazon_fr,France,
...,...,...,...,...,...,...,...,...
387,96,Pareja Del Ano,Sebastian Yatra X Myke Towers,https://charts-static.billboard.com/img/2021/0...,,Billboard,Monde,
388,97,Anyone,Justin Bieber,https://charts-static.billboard.com/img/2021/0...,,Billboard,Monde,
389,98,Prisoner,Miley Cyrus Featuring Dua Lipa,https://charts-static.billboard.com/img/2020/1...,,Billboard,Monde,
390,99,Woman,Doja Cat,https://charts-static.billboard.com/img/2018/0...,,Billboard,Monde,


In [24]:
result.to_csv('resultat_4_sites_new.csv', index=False)